# gameplay

> Fill in a module description here

In [ ]:
#| default_exp gameplay

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
%load_ext autoreload


In [ ]:
%autoreload 2

In [ ]:
#| export
import clue.model as model
import clue.ui as ui
from random import choice,shuffle,choices
import os

In [ ]:
#| hide
stgame = model.createClue(4)
stmoves = []

In [ ]:
#| export
def getPlay(pc,game, moves):
    
    

    ui.showBoard(pc,game,moves)
    playerRoom = model.roomOf(pc,game)
    passages = model.roomsAround(playerRoom)
    locs = [[3,x] for x in passages ]
    print("Move to Room:\n")
 
    l = ui.chooseKey(locs,pc,game,moves)
    l = passages[l-1]

    wKeys = [[1,x] for x in range(len(model.people))]
    p = ui.chooseKey(wKeys,1,game,moves) - 1

    wKeys = [[2,x] for x in range(len(model.weapons))]
    w = ui.chooseKey(wKeys,1,game,moves) - 1

    return p, w, l



In [ ]:
#| hide
stguess = getPlay(1,stgame,stmoves)
print(ui.showChoice(stguess))
def seenCounts(pc,game, moves):
    knoweledge = getSeen(pc,game, moves)
    p = [0,0,0]
    for x in knoweledge:
        i = int(x/100) -1
        p[i] += 1

    return p

NameError: name 'stgame' is not defined

In [ ]:
#| export




def simpleGuess(pc,game, moves):


    playerRoom = model.roomOf(pc,game)
    exclude  = set(model.getSeen(pc,game, moves))
    
    pdeck = [model.keyCard(1,x) for x in range(len(model.people))]
    pdeck = [x for x in  pdeck if x not in exclude]
    p = choice(pdeck)%100
    
    pdeck = [model.keyCard(2,x) for x in range(len(model.weapons))]
    pdeck = [x for x in pdeck if x not in exclude]
    w = choice(pdeck)%100
    
    pdeck = [model.keyCard(3,x) for x in model.roomsAround(playerRoom)]
    rdeck = [x for x in  pdeck if x not in exclude]
    
    if len(rdeck) == 0:
        l = pdeck[0]%100
    else:
        l = choice(rdeck)%100
    
    return (p,w,l)


In [ ]:
#| export

def itemsForType(t):
    if t == 1:
        return model.people
    if t == 2:
        return model.weapons
    return model.rooms
        
def itemGuess(t,moves):
    l = itemsForType(t)

    ret = [1 for x in range(len(l))]
    for x in moves:
        ret[x[t-1]] += 1
    #print(ret)
    return ret
    
def showCounts(t,moves):
    items = itemGuess(t,moves)
    base = itemsForType(t)
    #print(items,base)
    for i in range(len(items)):
        print(i,ui.cardFull(base[i]),items[i])


                              




In [ ]:
#| export
def rWalk(exclude,pc,game, moves):
    playerRoom = model.roomOf(pc,game)
    pdeck = [model.keyCard(3,x) for x in model.roomsAround(playerRoom)]
    rdeck = [x for x in  pdeck if x not in exclude]
    
    if len(rdeck) == 0:
        l = pdeck[0]%100
    else:
        l = choice(rdeck)%100
                              
    return l

In [ ]:
#| export


def roomsAway(roomNum,roomSize):
    foo = [x for x in range(roomSize)]
    middle = int( roomSize / 2)
    if roomNum <middle:
        lm = foo[roomNum - int(middle):]
        lSide = lm + foo[:roomNum] 
        lSide.reverse()
        rSide = foo[roomNum + 1:roomNum + 1 + middle]
    elif roomNum > middle:
        lSide = foo[roomNum-middle:roomNum]
        lSide.reverse()
        rSide = foo[roomNum +1: ] + foo[:roomNum-middle]
    else:
        lSide = foo[:middle]
        lSide.reverse()
        rSide = foo[middle+1:]
    return lSide, rSide

def repelWalk(exclude,pc,game, moves):
    playerRoom = model.roomOf(pc,game)
    lRooms, rRooms = roomsAway(playerRoom,len(model.rooms))
    #print("staring at " + ui.cardFull(model.rooms[playerRoom]))
    #print(playerRoom,lRooms,rRooms)
    #print(exclude)
    def score(rooms):
        rooms.reverse()
        weights = []
        cards = []
        for i in range(len(rooms)):
            r = rooms[i]
            cards.append(str(model.keyCard(3,r)) + " " + model.rooms[r][0])
            
            if model.keyCard(3,r) in exclude:
                w = 2**i
                #print(i,w)
                weights.append(w)
            else:
                weights.append(0)
        #print(cards,weights)
        return sum(weights)


    lPull = score(lRooms)
    rPull = score(rRooms)
    #trace.append([playerRoom,lPull,rPull])
    #print(trace)
    #ui.showBoard(pc,game,moves)
    achoice = model.roomsAround(playerRoom)
    if lPull > rPull:
        #print("to" + ui.cardFull(model.rooms[achoice[2]]))
        return achoice[2]
    if lPull < rPull:
        
        #print("to" + ui.cardFull(model.rooms[achoice[0]]))
        return achoice[0]
    #print("same")    
    return rWalk(exclude,pc,game, moves)


0 ([8, 7, 6, 5], [1, 2, 3, 4])
1 ([0, 8, 7, 6], [2, 3, 4, 5])
2 ([1, 0, 8, 7], [3, 4, 5, 6])
3 ([2, 1, 0, 8], [4, 5, 6, 7])
4 ([3, 2, 1, 0], [5, 6, 7, 8])
5 ([4, 3, 2, 1], [6, 7, 8, 0])
6 ([5, 4, 3, 2], [7, 8, 0, 1])
7 ([6, 5, 4, 3], [8, 0, 1, 2])
8 ([7, 6, 5, 4], [0, 1, 2, 3])


In [ ]:
#| export
def mimic(memory,walk):
    def betterGuess(pc,game, moves):

        exclude  = set(model.getSeen(pc,game, moves))

        if memory < 0:
            examine = []
        else:
            examine = moves[-memory:]


        keydeck = [model.keyCard(1,x) for x in range(len(model.people))]
        pdeck = [x for x in  keydeck if x not in exclude]
        pC = itemGuess(1,examine)
        pWeights = [pC[x%100] * pC[x%100] for x in  keydeck if x not in exclude]

        p = choices(pdeck,weights=pWeights)
        p = p[0]%100

        keydeck = [model.keyCard(2,x) for x in range(len(model.weapons))]
        pdeck = [x for x in keydeck if x not in exclude]
        pC = itemGuess(2,examine)
        pWeights = [pC[x%100] * pC[x%100] for x in  keydeck if x not in exclude]

        w = choices(pdeck,weights=pWeights)[0]%100
        
        l = walk(exclude,pc,game, moves)

        return (p,w,l)
    return betterGuess

In [ ]:
#| export
def cheater(walk):
    def betterGuess(pc,game, moves):

 
        exclude  = set(model.handOf(pc,game) +[x[-1] for x in moves] )

       
        examine = []
   


        keydeck = [model.keyCard(1,x) for x in range(len(model.people))]
        pdeck = [x for x in  keydeck if x not in exclude]
        pC = itemGuess(1,examine)
        pWeights = [pC[x%100] * pC[x%100] for x in  keydeck if x not in exclude]

        p = choices(pdeck,weights=pWeights)
        p = p[0]%100

        keydeck = [model.keyCard(2,x) for x in range(len(model.weapons))]
        pdeck = [x for x in keydeck if x not in exclude]
        pC = itemGuess(2,examine)
        pWeights = [pC[x%100] * pC[x%100] for x in  keydeck if x not in exclude]

        w = choices(pdeck,weights=pWeights)[0]%100
        
        l = walk(exclude,pc,game, moves)

        return (p,w,l)
    return betterGuess


def halfRight(walk):
    oracle = cheater(walk)
    
    def betterGuess(pc,game, moves):
        p,w,l = oracle(pc,game, moves)
        exclude  = model.handOf(pc,game) +[x[-1] for x in moves] 
        exclude = [x for x in exclude if x < 300]
        

        if len(exclude) < 0:
            return (p,w,l)

        wrong = choice(exclude)
        if wrong < 200:
            p = wrong%100
        else:
            w = wrong%100

        return (p,w,l)
    return betterGuess

## Visual

In [ ]:
#| export
def playerName(i):
    return ui.cardFull(model.cardLookup(1,i-1))
    
def runVisualGame( strats):
    numPlayers = len(strats)
    done = False
    agame = model.createClue(numPlayers)
    amoves = []
    player = 1
    lastM = ["" for x in range(numPlayers)]
   
             
    while not done:
        #print(seenCounts(player,agame,amoves))
        
        stratName, isUser, strat = strats[player - 1]
        
        if isUser:
            ui.printPlayerHand(player,agame)
            #showCounts(1,amoves)
            #showCounts(2,amoves)
            print(lastM[player-1])
        
        ans = strat(player,agame,amoves)
        if isUser:
            pass
            #os.system("clear")
        agame = model.relocatePlayers(player , ans[0],ans[2],agame)
        print( playerName(player) + " suspects " + ui.showChoice(ans)+".")
        check = model.getAnswer(player,agame,ans)
        
        if check[0] < 0:
            seen =  model.getSeen(player,agame, amoves)
            exclude  = set(seen)
            done = True
            for x in model.keysForGuess(ans):
                if x in exclude:
                    print("\n\n" + playerName(player) + " is not quite ready to accuse.")
                    done = False
            
        else:
            #(p,w,l,playerNum, c,y[0],y[1])
            card = check[-1]
            
            amoves.append(check)
            if isUser:
                msg = playerName(check[-2]) + " has " + ui.cardFull(model.keyLookUp(card))+"!"
            else:
                msg = playerName(check[-2]) + " responds."
            print(msg)
            lastM[player-1] = msg

        if not done:
            player = player + 1
            if player - 1 >=len(strats):
               player = 1 
    
    print("\n" + playerName(player) + " " + stratName + " won!")
    ui.printGame(agame) 

In [ ]:
#| hide
trace = []
p1 = ("player",True,getPlay)
s1 = ("simple",False,mimic(-1,rWalk))
s2 = ("simpleR",False,mimic(-1,repelWalk))
b1 = ("better",False,mimic(8,rWalk))
d1 =("deep",False,mimic(20,rWalk))
hr = ("halfRight",False,halfRight(rWalk))
strategies = [ p1,s2,hr,b1]
runVisualGame(strategies)

Hall, 🕯️ Candlestick, 🪠 Lead Pipe, 🔫 Revolver, Ball Room

╒═══════════╤═══════════════╤═════════════════╕
│ Ball Room │ Billiard Room │ 1) Conservatory │
│           │    W          │                 │
├───────────┼───────────────┼─────────────────┤
│ Study     │               │ 2) Dining Room  │
│    P      │               │    S            │
├───────────┼───────────────┼─────────────────┤
│ Library   │               │ 3) Hall         │
│    G      │               │                 │
├───────────┼───────────────┼─────────────────┤
│           │ Lounge        │ Kitchen         │
│           │    P          │    M            │
╘═══════════╧═══════════════╧═════════════════╛
Move to Room:

1 Conservatory
2 Dining Room
3*Hall


Choose from 1 to 3:  2


1 👠 Miss Scarlet
2 🧐 Colonel Mustard
3 ♡  Mrs. White
4 🐉 Mr. Green
5 🍆 Professor Plum
6 🦚 Mrs. Peacock


Choose from 1 to 6:  3


1 🔪 Knife
2*🔫 Revolver
3 🪢 Rope
4 🔧 Wrench
5*🕯️ Candlestick
6*🪠 Lead Pipe


...

1 👠 Miss Scarlet
2 🧐 Colonel Mustard
3 ♡  Mrs. White
4 🐉 Mr. Green
5*🍆 Professor Plum        🧐
6 🦚 Mrs. Peacock


Choose from 1 to 6:  3


1*🔪 Knife
2*🔫 Revolver              ♡, 🐉, 🍆
3*🪢 Rope                  🧐
4 🔧 Wrench
5*🕯️ Candlestick          ♡, 🐉, 🍆
6*🪠 Lead Pipe             🐉, 🍆


Choose from 1 to 6:  4


👠 Miss Scarlet suspects ♡  Mrs. White with the 🔧 Wrench in the Kitchen.
🐉 Mr. Green has ♡  Mrs. White!
🧐 Colonel Mustard suspects 🐉 Mr. Green with the 🔧 Wrench in the Lounge.
🐉 Mr. Green responds.
♡  Mrs. White suspects 🐉 Mr. Green with the 🪢 Rope in the Kitchen.


 gave a false one 🪢 Rope!
♡  Mrs. White suspects 🍆 Professor Plum with the 🔧 Wrench in the Kitchen.


 gave a false one 🍆 Professor Plum!
♡  Mrs. White suspects 🧐 Colonel Mustard with the 🪢 Rope in the Kitchen.
🐉 Mr. Green responds.
🐉 Mr. Green suspects 🐉 Mr. Green with the 🔧 Wrench in the Kitchen.

🐉 Mr. Green better won!
Solution
🐉 Mr. Green, 🔧 Wrench, Kitchen

          [1] 👠 Miss Scarlet is in Kitchen          
Hall, 🕯️ Candlestick, 🪠 Lead Pipe, 🔫 Revolver, Ball Room

        [2] 🧐 Colonel Mustard is in Kitchen         
Conservatory, Library, Study, 🔪 Knife, Dining Room

          [3] ♡  Mrs. White is in Kitchen           
👠 Miss Scarlet, 🪢 Rope, Billiard Room, 🍆 Professor Plum

           [4] 🐉 Mr. Green is in Kitchen  

In [ ]:
#| export
def runGame( strats):
    numPlayers = len(strats)
    done = False
    agame = model.createClue(numPlayers)
    amoves = []
    player = 1
          
    while not done:
        #print(seenCounts(player,agame,amoves))
        
        stratName, isUser, strat = strats[player - 1]
        
        ans = strat(player,agame,amoves)
        agame = model.relocatePlayers(player , ans[0],ans[2],agame)
        check = model.getAnswer(player,agame,ans)
        
        if check[0] < 0:
            seen =  model.getSeen(player,agame, amoves)
            exclude  = set(seen)
            done = True
            for x in model.keysForGuess(ans):
                if x in exclude:
                    done = False
            
        else:
            #(p,w,l,playerNum, c,y[0],y[1])
            card = check[-1]
            
            amoves.append(check)

        if not done:
            player = player + 1
            if player - 1 >= numPlayers:
               player = 1 
    
    return player, agame , amoves


In [ ]:
#| hide
s1 = ("simple",False,mimic(-1,rWalk))
b1 = ("better",False,mimic(2,rWalk))
d1 =("deep",False,mimic(8,rWalk))
c1 = ("cheater",False,cheater(repelWalk))
s2 = ("simpleR",False,mimic(4,rWalk))
hr = ("halfRight",False,halfRight(rWalk))

strategies = [s1, s2,b1,b1,d1,c1]
strategies = [s1, s2,s1,hr]
counts = [0 for x in range(len(strategies))]
for c  in range(100):
    p, g, m = runGame(strategies)
    counts[p-1] += 1

m = zip([x[0] for x in strategies],counts)
for x in m:
    print(x)

('simple', 35)
('simpleR', 44)
('simple', 21)
('halfRight', 0)


In [ ]:
#| hide
chooseCard(rooms,cardFull)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()